In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import gc
import glob

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False


# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [3]:
# data_type = "train"
# month = "07"
# category = "잔액정보"

# local
root_path = '../../data/야구데이터'

# colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

drive_folder = f'{root_path}/스탯티즈_Raw_data/'

In [5]:
df1 = pd.read_csv(f'{drive_folder}FA투수데이터 1차.csv', encoding='utf-8')
df1

,FA년도,구단명,선수명,세부 포지션,투,인정년수,구분,FA등급,플레이년도,나이,...,구원 이닝,선발 WAR,구원 WAR,종합 WAR,연봉,비고1,비고2,잔류 여부,FA 계약 연수,FA 계약 총액
0,2011,롯데,강영식,RP,좌투,12.0,자격유지,A,2010,29.0,...,NaN,NaN,1.09,1.09,12000.0,고졸,NaN,NaN,NaN,NaN
1,2012,롯데,강영식,RP,좌투,12.0,자격유지,A,2011,30.0,...,NaN,NaN,1.26,1.26,30000.0,고졸,NaN,NaN,NaN,NaN
2,2013,롯데,강영식,RP,좌투,12.0,자격유지,A,2012,31.0,...,NaN,NaN,0.66,0.66,30000.0,고졸,NaN,NaN,NaN,NaN
3,2014,롯데,강영식,RP,좌투,12.0,자격유지,A,2013,32.0,...,39.2,NaN,0.49,0.49,30000.0,고졸,NaN,잔류,4.0,17.0
4,2015,롯데,강영식,RP,좌투,NaN,FA 1년차,NaN,2014,33.0,...,47.0,NaN,1.18,1.18,30000.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,2021,두산,홍건희,RP,우투,8.0,신규,A,2020,28.0,...,68.2,NaN,0.22,0.22,5300.0,고졸,NaN,NaN,NaN,NaN
323,2022,두산,홍건희,RP,우투,8.0,신규,A,2021,29.0,...,74.1,NaN,2.67,2.67,11000.0,고졸,NaN,NaN,NaN,NaN
324,2023,두산,홍건희,CP,우투,8.0,신규,A,2022,30.0,...,62.0,NaN,0.49,0.49,25000.0,고졸,NaN,NaN,NaN,NaN
325,2024,두산,홍건희,CP,우투,8.0,신규,A,2023,31.0,...,61.2,NaN,1.23,1.23,30000.0,고졸,NaN,잔류,4.0,24.5


#### 우진님께서 1차로 "TEAM"이 들어간 컬럼, 구단명이 "은퇴"인 row 삭제 진행

In [8]:
# 출력 옵션 조정 (생략 없이 dtypes 포함 모든 열 보기)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

### 🔎 결측치 확인

In [11]:
print("🔍 [결측치 개수]")
print("▶ df1:\n", df1.isnull().sum().sort_values(ascending=False))

🔍 [결측치 개수]
▶ df1:
 비고2         324
FA 계약 총액    264
FA 계약 연수    264
잔류 여부       257
선발 WAR      171
2루타허용        53
선발 이닝        53
구원 이닝        53
FA등급         53
비고1          53
인정년수         53
3루타허용        53
구원 WAR       48
블론세이브        36
블론홀드         36
연봉            7
FIP           4
이닝수           4
종합 WAR        0
볼넷허용          0
ERA           0
폭투            0
탈삼진           0
사구허용          0
FA년도          0
피홈런           0
게임수           0
선수명           0
세부 포지션        0
투             0
구분            0
플레이년도         0
나이            0
선발등판          0
구단명           0
구원등판          0
승리            0
패전            0
세이브           0
홀드            0
자책점           0
피안타           0
dtype: int64


In [13]:
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')

In [15]:
# 둘다 결측치가 대부분이고, 분석에 영향이 크지 않아보여 삭제
df1 = df1.drop(columns=['비고2'], errors='ignore')

In [17]:
# "TEAM"이 들어간 컬럼에 결측치가 많다.
# 일단 정말 필요한 컬럼들의 전처리를 위해 삭제하고 진행
# 필요시 나중에 따로 전처리 하는걸로
# df1 = df1.loc[:, ~df1.columns.str.contains("TEAM", case=False, na=False)]

In [55]:
# excel로 살펴보기 해해
# df1.to_excel("KBO FA 투수 [2013-2024]_1차_전처리.xlsx", index=False)

In [19]:
print("🔍 [결측치 개수]")
print("▶ df1:\n", df1.isnull().sum().sort_values(ascending=False))

🔍 [결측치 개수]
▶ df1:
 FA 계약 총액    264
FA 계약 연수    264
잔류 여부       257
선발 WAR      171
FA등급         53
2루타허용        53
구원 이닝        53
선발 이닝        53
인정년수         53
비고1          53
3루타허용        53
구원 WAR       48
블론세이브        36
블론홀드         36
연봉            7
FIP           4
이닝수           4
종합 WAR        0
피홈런           0
ERA           0
폭투            0
탈삼진           0
사구허용          0
볼넷허용          0
FA년도          0
피안타           0
구단명           0
홀드            0
세이브           0
패전            0
승리            0
구원등판          0
선발등판          0
게임수           0
나이            0
플레이년도         0
구분            0
투             0
세부 포지션        0
선수명           0
자책점           0
dtype: int64


In [21]:
df1['구단명'].value_counts()

구단명
삼성     66
한화     44
LG     39
롯데     37
넥센     25
SK     24
두산     24
KIA    21
NC     17
키움     14
KT     12
SSG     3
우리      1
Name: count, dtype: int64

#### 우리 --> 마일영 의 기록은 2010년도부터 한화로 되어있어서 한화로 교체
#### 넥센 --> 키움
#### SK --> SSG

In [24]:
# 팀명 변경 딕셔너리
team_rename_map = {
    '넥센': '키움',
    'SK': 'SSG',
}

# 1. 일반적인 팀명 변경 적용
df1['구단명'] = df1['구단명'].replace(team_rename_map)

# 2. 마일영 선수의 2010년도 기록만 예외적으로 '한화'로 수정
df1.loc[(df1['선수명'] == '마일영') & (df1['FA년도'] == 2010), '구단명'] = '한화'

In [26]:
df1['구단명'].value_counts()

구단명
삼성     66
한화     45
키움     39
LG     39
롯데     37
SSG    27
두산     24
KIA    21
NC     17
KT     12
Name: count, dtype: int64

In [36]:
df1['비고1'].unique()

array(['고졸', nan, '신규 35세 23년 C등급', '35세 이상', '대졸', '신규 35세 해외 복귀',
       '23년 B등급'], dtype=object)

#### 비고1은 결측치 바로 위에값으로 결측치를 처리해준다.

In [41]:
df1['비고1'] = df1['비고1'].ffill()

In [51]:
df1['비고1'].value_counts()

비고1
고졸                242
대졸                 50
35세 이상             15
신규 35세 23년 C등급     10
신규 35세 해외 복귀        5
23년 B등급             5
Name: count, dtype: int64

In [54]:
df1['이닝수'].isna().sum()

4

#### 이닝수 = 선발 이닝 + 구원 이닝 ---> (일단 값이 정확하진 않지만 결측치 채우고 추후 생각)

In [57]:
df1['이닝수'] = df1['이닝수'].fillna(df1['선발 이닝'] + df1['구원 이닝'])

In [59]:
df1['이닝수'].isna().sum()

0

In [61]:
df1['FIP'].isna().sum()

4

#### FIP : 수비 무관 평균 자책점
#### 지금 데이터엔, FIP가 전년도 것으로 써져있는 것 같음 (스탯티즈)
#### --> 알고보니 다 전년도 기록임 (ex : 스탯티즈의 2013년 -- 데이터의 2014년)
#### 결측치인 부분은 그 선수가 이적을 해서, FIP가 2개
#### 즉, 다른 FIP는 1년 기준이지만 결측치인 FIP는 1년에 값이 2개 나와있음 그래서 결측치인 것

In [66]:
df1['블론홀드'].isna().sum()

36

In [68]:
df1['블론세이브'].isna().sum()

36

#### 블론홀드나 세이브의 결측치는 스탯티즈에 값이 없는 값이라 결측치이다.
#### -> 0으로 처리해도 되려나 ?? : 이미 기록이 없는데 0으로 들어간 값도 있기 때문
#### -> 기록이 없는게 진짜 기록이 0인게 맞나?

In [80]:
# 선발 WAR = 종합 WAR - 구원 WAR
df1['선발 WAR'].isna().sum()

171

In [82]:
# 구원 WAR = 종합 WAR - 선발 WAR
df1['구원 WAR'].isna().sum()

48

In [84]:
df1['종합 WAR'].isna().sum()

0

#### 종합 WAR = 선발 WAR + 구원 WAR 

In [86]:
# 구원 WAR 결측치 채우기
df1['구원 WAR'] = df1['구원 WAR'].fillna(df1['종합 WAR'] - df1['선발 WAR'])

In [90]:
# 선발 WAR 결측치 채우기
df1['선발 WAR'] = df1['선발 WAR'].fillna(df1['종합 WAR'] - df1['구원 WAR'])

In [94]:
df1['선발 이닝'].isna().sum()

53

In [96]:
df1['구원 이닝'].isna().sum()

53

#### 마찬가지로 선발 이닝과 구원 이닝이 결측치인 것은
#### 기록이 없어서 결측치로 되어있다. -> 어떻게 처리?